<a href="https://colab.research.google.com/github/JeanMusenga/PhD-Thesis_2024_Musenga/blob/main/TextCNN_with_GloVe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://chatgpt.com/share/21220a6d-4cca-48bc-b461-bca6b0e140bc

In [ ]:
pip install tensorflow

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [6]:
file_path = './saved_file'
file_path = ('ARPs_and_ProgrammingPosts.xlsx')
arp_data = pd.read_excel(file_path, sheet_name='13,205 ARPs')


In [36]:
import os

# List the files in the upload directory
uploaded_files = os.listdir('/mnt/data')

uploaded_files

['glove']

In [40]:
import zipfile

# Verify the correct file name
zip_file_path = '/mnt/data/GloVe-master.zip'  # Use the correct file name here
extract_dir = '/mnt/data/glove'

In [41]:
# Unzipping the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Extraction completed.")

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/GloVe-master.zip'

In [7]:
# Parameters
max_words = 10000  # Maximum number of words to keep, based on word frequency
max_len = 100      # Maximum length of all sequences
embedding_dim = 100  # Dimension of the GloVe embeddings

In [8]:
# Extract the text and labels
texts = arp_data['Question_body'].astype(str).tolist()
labels = arp_data['Label'].tolist()

In [9]:
# Tokenize the text
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [10]:
# Pad the sequences to ensure uniform input size
data = pad_sequences(sequences, maxlen=max_len)

In [11]:
# Convert labels to numpy array
labels = tf.keras.utils.to_categorical(labels, num_classes=2)

In [12]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


In [13]:
# Load GloVe embeddings
def load_glove_embeddings(glove_file_path, embedding_dim):
    embeddings_index = {}
    with open(glove_file_path, 'r', encoding='utf8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

In [14]:
# Path to the GloVe file (update this to your GloVe file path)
glove_file_path = 'path_to_your_glove_file/glove.6B.100d.txt'
embeddings_index = load_glove_embeddings(glove_file_path, embedding_dim)


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_your_glove_file/glove.6B.100d.txt'

In [ ]:
# Create an embedding matrix
word_index = tokenizer.word_index
vocab_size = min(max_words, len(word_index) + 1)
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in word_index.items():
    if i >= max_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all zeros
        embedding_matrix[i] = embedding_vector


In [ ]:
# Define the neural network model
def create_model(vocab_size, embedding_dim, input_length, embedding_matrix):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size,
                                  output_dim=embedding_dim,
                                  weights=[embedding_matrix],
                                  input_length=input_length,
                                  trainable=False),
        tf.keras.layers.Conv1D(128, 5, activation='relu'),
        tf.keras.layers.GlobalMaxPooling1D(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Create the model
model = create_model(vocab_size, embedding_dim, max_len, embedding_matrix)


In [ ]:
# Train the model
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

In [ ]:
# Make predictions
y_pred = model.predict(x_test)
y_pred_classes = y_pred.argmax(axis=1)
y_true_classes = y_test.argmax(axis=1)

In [ ]:
# Calculate precision, recall, and F1-score
report = classification_report(y_true_classes, y_pred_classes, target_names=['Class 0', 'Class 1'])
print(report)


In [ ]:
# Save the model
model.save('glove_textcnn_model.h5')